In [ ]:

import matplotlib.pyplot as plt # là thư viện để vẽ đồ thị, module sử dụng là pyplot, có chức năng như Matlab nhưng dung python
import numpy as np # linear algebra(numeric python, là thư viện toán học được tối ưu hóa)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)được phát triển lên từ np dùng để xử lý dữ liệu 
import seaborn as sns#tạo các hình ảnh trực quan, được phát triển lên từ matplotlib
%matplotlib inline
sns.set()
from sklearn.model_selection import train_test_split# thư viện sklearn(scikit learn) dùng để mo hình hóa dữ liệu, xử lý các thuật toán trong ml
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


In [ ]:
train_data = pd.read_csv('../input/titanic/train.csv')#ĐỌC DATA 
test_data = pd.read_csv('../input/titanic/test.csv')



In [ ]:
train_data.head()#SHOW  GIAO DIỆN DATA, 5 CÁI ĐẦU TIÊN


In [ ]:
train_data.info()

In [ ]:
test_data.head()#BỘ TEST DATA ĐÃ LOẠI SURVIDE 


In [ ]:
test_data.info()

In [ ]:
#PassengerId, Name and Ticket are not concern to survived
# drop unuseful columns
train_data.drop(['PassengerId', 'Name', 'Ticket'], axis= 1, inplace = True)# DÒNG LỆNH LOẠI BỎ CÁC CỘT
train_data.head()# XEM LẠI DATA SAU KHI XÓA CÁC CỘT KHÔNG CẦN

To consider Cabin's columns

In [ ]:
train_data.Cabin.describe()# CABIN LÀ MỘT THUỘC TÍNH TRONG BỘ TRAIN_DATA

In [ ]:
train_data.Cabin.isnull().sum()# KIỂM TRA XEM TRONG CỘT CABIN NÓ THIẾU BAO NHIÊU GIÁ TRỊ, IN RA

We can see that the missing values is too large. It is arround three devide four data of comlum so we have to drop this column to avoid bad effect to train process

In [ ]:
train_data.drop('Cabin', axis= 1, inplace = True)# LOẠI BỎ CỘT NÀY RA KHỎI DATA_TRAIN
train_data.head()

After removed some columns not related we have dataset like this, however there are still columns that loss data values so we next to handle missing values


To begin with Age's columns

In [ ]:
train_data.Age.describe()

In [ ]:
# Check Age Distribution
train_data.Age.hist();# TẠO BIỂU ĐỒ CHO CỘT AGE CỦA TRAIN_DATA

Because the mean and median valuse are very close so we will choose mean to fill the missing values

In [ ]:
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)# THÊM VÀO CÁC GIÁ TRỊ BỊ THIẾU BẰNG GIÁ TRỊ MEAN CỦA THUỘC TÍNH AGE


After fill the missing values in Age's column we get:

In [ ]:
train_data.Age.isnull().sum() # KIỂM TRA XEM CÒN THIẾU HAY KHÔNG, IN RA

In [ ]:
train_data.Age.hist();

In [ ]:
train_data.loc[(train_data.Age < 18),'AgeGroup'] = 'Children'
train_data.loc[(train_data.Age >=18) & (train_data.Age <30),'AgeGroup'] = 'Youth'
train_data.loc[(train_data.Age >=30) & (train_data.Age <60),'AgeGroup'] = 'Adult'
train_data.loc[(train_data.Age >=60),'AgeGroup'] = 'Senior'
train_data.head()

In [ ]:
train_data.Age.describe()

To make the same with Embarked column

In [ ]:
train_data.Embarked.unique()

In [ ]:
train_data.Embarked.isnull().sum()

In [ ]:
train_data.Embarked.value_counts()

In [ ]:
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace = True)
train_data['Embarked'].isnull().sum()

In [ ]:
train_data['Embarked'].replace({'C':'Cherbourg','Q':'Quuentowns','S':'Southamton'},inplace = True)# THAY THẾ CÁC CHỮ CÁI CQS BẰNG CÁC TÊN CỦA CẢNG
train_data.head()

We have dataset after fixed look like that:

In [ ]:
train_data.head()

To make it easy, I will change Fare into 3 level are : high, medium and low

In [ ]:
train_data.loc[(train_data.Fare < 50),'FareGroup'] = 'Low'
train_data.loc[(train_data.Fare >= 50) & (train_data.Fare < 100),'FareGroup'] = 'Medium'
train_data.loc[(train_data.Fare >= 100),'FareGroup'] = 'High'
train_data.head()

In [ ]:
train_data.info()

In [ ]:
def explore_feature(feature):
    group= train_data.groupby(feature).mean()
    plots = group.Survived.sort_values().plot(kind = 'bar', fontsize=15)
    plots.title.set_size(40);
    plt.xlabel(feature, fontsize=15);
    plt.ylabel('% Survived', fontsize=15);
    plt.title("% Survived according to {} feature\n".format(feature), fontsize=20)
    for bar in plots.patches:#vẽ biểu đồ, 
        plots.annotate(format(bar.get_height(), '.2f'), 
                       (bar.get_x() + bar.get_width() / 2, 
                        bar.get_height()), ha='center', va='center',
                       size=15, xytext=(0, 8),
                       textcoords='offset points')
  # Using Matplotlib's annotate function and
  # passing the coordinates where the annotation shall be done
  # x-coordinate: bar.get_x() + bar.get_width() / 2
  # y-coordinate: bar.get_height()
  # free space to be left to make graph pleasing: (0, 8)
  # ha and va stand for the horizontal and vertical alignment

Pclass Exploration

In [ ]:
explore_feature('Pclass')

So, it is obvious that the higher your class level on Titanic, the higher probability to be survived

Sex Exploration

In [ ]:
explore_feature('Sex')

 it is obvious that females on Titanic had higher probability than males to be survived

Age exploration

 It is obvious that females on Titanic had higher probability than males to be survived

In [ ]:
explore_feature('AgeGroup')

So, the children (Age < 18) had the higher probability to survive, whereas Seniors (Age > 60) had the lower probability

In [ ]:
explore_feature('Embarked')

People from Cherbourg have more chances than from Queenstown and Southampton

In [ ]:
explore_feature('SibSp')

Excluding the single passenger, the less No. of siblings/spouses, the more chance to survive

DATA PROCESSING

Splitting Training data into Features and Target


In [ ]:
features= train_data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
target = train_data['Survived']
print(features)

In [ ]:
features = pd.get_dummies(features)
features.head()

In [ ]:
features.info()

Splitting Features and Target data into training set and test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)

SCALING

In [ ]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate

models=[LogisticRegression(),
        DecisionTreeClassifier(),
        KNeighborsClassifier()]
for model in models:
    # fit the model with cross validation
    results=cross_validate(model,X_train_sc,y_train,cv=10)
    # accuracy
    r2=results['test_score'].mean()
    # print the result
    m=str(model)
    print('r2 for',m[:m.index('(')],'=',r2)

In [ ]:
print(results)

In [ ]:
# Logistic Regression

param_lr={'penalty':['l1','l2'],
          'C' : [0.01,0.1,1,10,50,100,200,300],
         'solver':['liblinear', 'saga']}

gs_lr = GridSearchCV(LogisticRegression(),param_grid = param_lr, scoring="accuracy",n_jobs=-1)
gs_lr.fit(X_train_sc,y_train)
best_lr=gs_lr.best_estimator_
print(best_lr)
print('score=',gs_lr.best_score_)

In [ ]:
# Decision Tree
param_dt={'max_features': [1, 2, 3, 5, 6, 7, 8, 9, 10, 15],
          'min_samples_split': [2, 3, 4, 5, 6, 7, 10, 15],
          'min_samples_leaf': [1, 2, 3, 5, 6, 7, 8, 10, 15],
          'splitter':['best']}

gs_dt = GridSearchCV(DecisionTreeClassifier(),param_grid = param_dt, scoring="accuracy",n_jobs=-1)
gs_dt.fit(X_train_sc,y_train)
best_dt=gs_dt.best_estimator_
print(best_dt)
print('score=',gs_dt.best_score_)

In [ ]:
# KNN
param_kn={'n_neighbors':[1,2,3,5,7,10,14,15]}

gs_kn = GridSearchCV(KNeighborsClassifier(),param_grid = param_kn, scoring="accuracy",n_jobs=-1)
gs_kn.fit(X_train_sc,y_train)
best_kn=gs_kn.best_estimator_
print(best_kn)
print('score=',gs_kn.best_score_)